In [1]:
# mysql connection
import mysql.connector

mydb = mysql.connector.connect(
  host="cowstudio.wayne-lee.cn",
  user="cowstudio",
  password="cowstudio_2119",
  database="cowstudio"
)

In [2]:
# get spark session, 2g mem per executor
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
import os

# set python env
os.environ['PYSPARK_PYTHON'] = "/opt/conda3/envs/lab2/bin/python"
spark = SparkSession.builder \
    .appName("ItemODS") \
    .master("spark://node01:10077") \
    .enableHiveSupport()\
    .config("spark.driver.memory", "1g") \
    .config("spark.executor.memory", "1g") \
    .config("spark.cores.max", "1") \
    .config("spark.sql.shuffle.partitions", "12") \
    .config("spark.sql.autoBroadcastJoinThreshold", "-1") \
    .getOrCreate()

sc = spark.sparkContext

23/04/21 08:52:30 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/04/21 08:52:32 WARN util.Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/04/21 08:52:32 WARN util.Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
23/04/21 08:52:32 WARN spark.SparkContext: Please ensure that the number of slots available on your executors is limited by the number of cores to task cpus and not another custom resource. If cores is not the limiting resource then dynamic allocation will not work properly!


In [3]:
# get all items from db, and create an rdd
from datetime import datetime
date_string = datetime.today().strftime('%Y-%m-%d')
cur = mydb.cursor()
cur.execute(f"SELECT items.*,{date_string} as date from items")
result = cur.fetchall()
all_items = sc.parallelize(result)
print(all_items.count())

433


In [4]:
from pyspark.sql.types import DecimalType, TimestampType, StructType, StructField, IntegerType, StringType

schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("description", StringType(), True),
    StructField("category", StringType(), True),
    StructField("price", DecimalType(10, 2), True),
    StructField("stock", IntegerType(), True),
    StructField("province", StringType(), True),
    StructField("city", StringType(), True),
    StructField("district", StringType(), True),
    StructField("image_url", StringType(), True),
    StructField("user_id", IntegerType(), True),
    StructField("user_type", StringType(), True),
    StructField("created_at", TimestampType(), True),
    StructField("specific_attributes", StringType(), True),
    StructField("date", StringType(), True) 
])
item_df = all_items.toDF(schema)

In [5]:
# spark.sql("drop table if exists item_ods").show()

++
||
++
++



In [6]:
item_df.write.mode("overwrite").partitionBy("date").saveAsTable("item_ods")

23/04/21 08:53:10 WARN session.SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.


In [7]:
spark.sql("show tables").show()

+--------+---------+-----------+
|database|tableName|isTemporary|
+--------+---------+-----------+
| default| item_ods|      false|
| default|     test|      false|
| default|    test2|      false|
| default| user_ods|      false|
+--------+---------+-----------+



In [8]:
spark.stop()
mydb.close()